In [ ]:
# 多输入通道和多输出通道

# 对于真实数据的维度经常更高，例如彩色图像在高和宽两个维度外还有RGB（红绿蓝）3个通道
# 包含多个输入输出通道的卷积核


In [8]:
from imp import reload
import torch
from torch import nn
import sys

sys.path.append("..")
import d2l_pytorch.d2l as d2l
reload(d2l)




<module 'd2l_pytorch.d2l' from '..\\d2l_pytorch\\d2l.py'>

In [11]:
def corr2d_multi_in(X, K):
  # 沿着X和K的第0维（通道维度）分别计算再想加
  res = d2l.corr2d(X[0, :, :], K[0, :, :])
  for i in range(1, X.shape[0]):
    res += d2l.corr2d(X[i, :, :], K[i, :, :])

  return res


In [21]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]], [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]],[[1, 2], [3, 4]]])

corr2d_multi_in(X,K)


tensor([[ 56.,  72.],
        [104., 120.]])

In [ ]:
# 多输出通道不论输出通道是多少，输出通道都为1


In [22]:
def corr2d_multi_in_out(X, K):
  return torch.stack([corr2d_multi_in(X, k) for k in K])

In [23]:
K = torch.stack([K, K+1, K+2])
K.shape

torch.Size([3, 2, 2, 2])

In [24]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])